## IMDB TMDB Movie Dataset Data Processing

### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow import keras
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import ast
import datetime
import sklearn
import scipy
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
tqdm.pandas()

In [3]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  

In [4]:
df  = pd.DataFrame(pd.read_csv('../data/IMDB TMDB Movie Metadata Big Dataset (1M).csv'))
df.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,Star3,Star4,Writer,Director_of_Photography,Producers,Music_Composer,genres_list,Cast_list,overview_sentiment,all_combined_keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Elliot Page,Ken Watanabe,Christopher Nolan,Wally Pfister,"Thomas Tull, Christopher Nolan, Chris Brigham,...",Hans Zimmer,"['Action', 'Science Fiction', 'Adventure']","['Tim Kelleher', 'Silvie Laguna', 'Natasha Bea...",-0.011111,"['s', 'philosophy', 'skilled', 'kidnapping', '..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Jessica Chastain,Mackenzie Foy,Jonathan Nolan,Hoyte van Hoytema,"Jake Myers, Emma Thomas, Jordan Goldberg, Thom...",Hans Zimmer,"['Adventure', 'Drama', 'Science Fiction']","['Jeff Hephner', 'William Devane', 'Elyes Gabe...",0.045455,"['thoughtful', 'use', 'scientist', 'quantum me..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,Aaron Eckhart,Michael Caine,Jonathan Nolan,Wally Pfister,"Kevin De La Noy, Thomas Tull, Christopher Nola...","Hans Zimmer, James Newton Howard","['Drama', 'Action', 'Crime', 'Thriller']","['Tommy Lister Jr.', 'Edison Chen', 'Beatrice ...",0.025000,"['reign', 'harvey', 'proves', 'partnership', '..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Sigourney Weaver,Michelle Rodriguez,James Cameron,Mauro Fiore,"Jon Landau, Colin Wilson, Laeta Kalogridis, Ja...",James Horner,"['Action', 'Adventure', 'Fantasy', 'Science Fi...","['Carvon Futrell', 'Joel David Moore', 'Jon Cu...",0.041667,"['space war', 'civilization', 'society', 'mari..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,Scarlett Johansson,Jeremy Renner,Sydney Newman,NaN,NaN,NaN,"['Science Fiction', 'Action', 'Adventure']","['Haneyuri', 'Nako Mizusawa', 'Marin', 'Rikako...",0.025000,"['s', 'world', 'begins', 'e', 'global safety',..."


In [5]:
df.shape

(1072255, 42)

In [6]:
df.iloc[:,:20].head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,tagline,production_companies
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,160000000,https://www.warnerbros.com/movies/inception,tt1375666,en,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Legendary Pictures, Syncopy, Warner Bros. Pict..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,165000000,http://www.interstellarmovie.net/,tt0816692,en,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Legendary Pictures, Syncopy, Lynda Obst Produc..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,185000000,https://www.warnerbros.com/movies/dark-knight/,tt0468569,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"DC Comics, Legendary Pictures, Syncopy, Isobel..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,237000000,https://www.avatar.com/movies/avatar,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Dune Entertainment, Lightstorm Entertainment, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,220000000,https://www.marvel.com/movies/the-avengers,tt0848228,en,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,Marvel Studios


In [7]:
df.iloc[:,20:40].head()

,production_countries,spoken_languages,keywords,release_year,Director,AverageRating,Poster_Link,Certificate,IMDB_Rating,Meta_score,Star1,Star2,Star3,Star4,Writer,Director_of_Photography,Producers,Music_Composer,genres_list,Cast_list
0,"United Kingdom, United States of America","English, French, Japanese, Swahili","['rescue', 'mission', 'dream', 'airplane', 'pa...",2010.0,Christopher Nolan,8.8,https://m.media-amazon.com/images/M/MV5BMjAxMz...,UA,8.8,74.0,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,Christopher Nolan,Wally Pfister,"Thomas Tull, Christopher Nolan, Chris Brigham,...",Hans Zimmer,"['Action', 'Science Fiction', 'Adventure']","['Tim Kelleher', 'Silvie Laguna', 'Natasha Bea..."
1,"United Kingdom, United States of America",English,"['rescue', 'future', 'spacecraft', 'race again...",2014.0,Christopher Nolan,8.7,https://m.media-amazon.com/images/M/MV5BZjdkOT...,UA,8.6,74.0,Matthew McConaughey,Anne Hathaway,Jessica Chastain,Mackenzie Foy,Jonathan Nolan,Hoyte van Hoytema,"Jake Myers, Emma Thomas, Jordan Goldberg, Thom...",Hans Zimmer,"['Adventure', 'Drama', 'Science Fiction']","['Jeff Hephner', 'William Devane', 'Elyes Gabe..."
2,"United Kingdom, United States of America","English, Mandarin","['joker', 'sadism', 'chaos', 'secret identity'...",2008.0,Christopher Nolan,9.0,https://m.media-amazon.com/images/M/MV5BMTMxNT...,UA,9.0,84.0,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,Jonathan Nolan,Wally Pfister,"Kevin De La Noy, Thomas Tull, Christopher Nola...","Hans Zimmer, James Newton Howard","['Drama', 'Action', 'Crime', 'Thriller']","['Tommy Lister Jr.', 'Edison Chen', 'Beatrice ..."
3,"United States of America, United Kingdom","English, Spanish","['future', 'society', 'culture clash', 'space ...",2009.0,James Cameron,7.9,https://m.media-amazon.com/images/M/MV5BMTYwOT...,UA,7.8,83.0,Sam Worthington,Zoe Saldana,Sigourney Weaver,Michelle Rodriguez,James Cameron,Mauro Fiore,"Jon Landau, Colin Wilson, Laeta Kalogridis, Ja...",James Horner,"['Action', 'Adventure', 'Fantasy', 'Science Fi...","['Carvon Futrell', 'Joel David Moore', 'Jon Cu..."
4,United States of America,"English, Hindi, Russian","['new york city', 'superhero', 'shield', 'base...",2012.0,Joss Whedon,8.0,https://m.media-amazon.com/images/M/MV5BNDYxNj...,UA,8.0,69.0,Robert Downey Jr.,Chris Evans,Scarlett Johansson,Jeremy Renner,Sydney Newman,NaN,NaN,NaN,"['Science Fiction', 'Action', 'Adventure']","['Haneyuri', 'Nako Mizusawa', 'Marin', 'Rikako..."


In [8]:
df.iloc[:,40:].head()

,overview_sentiment,all_combined_keywords
0,-0.011111,"['s', 'philosophy', 'skilled', 'kidnapping', '..."
1,0.045455,"['thoughtful', 'use', 'scientist', 'quantum me..."
2,0.025000,"['reign', 'harvey', 'proves', 'partnership', '..."
3,0.041667,"['space war', 'civilization', 'society', 'mari..."
4,0.025000,"['s', 'world', 'begins', 'e', 'global safety',..."


In [9]:
df.dtypes

id                           int64
title                       object
vote_average               float64
vote_count                   int64
status                      object
release_date                object
revenue                      int64
runtime                      int64
adult                         bool
backdrop_path               object
budget                       int64
homepage                    object
imdb_id                     object
original_language           object
original_title              object
overview                    object
popularity                 float64
poster_path                 object
tagline                     object
production_companies        object
production_countries        object
spoken_languages            object
keywords                    object
release_year               float64
Director                    object
AverageRating              float64
Poster_Link                 object
Certificate                 object
IMDB_Rating         

### Type Conversion

In [10]:
type(df['release_date'][0])

str

`release_date` is in `str` format. Let's change that to its original format.

In [11]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
print(type(df['release_date'][0]))
print(df['release_date'].head())

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
0   2010-07-15
1   2014-11-05
2   2008-07-16
3   2009-12-15
4   2012-04-25
Name: release_date, dtype: datetime64[ns]


In [12]:
array_df = df[['keywords', 'genres_list', 'Cast_list', 'all_combined_keywords']]
array_df.head()

,keywords,genres_list,Cast_list,all_combined_keywords
0,"['rescue', 'mission', 'dream', 'airplane', 'pa...","['Action', 'Science Fiction', 'Adventure']","['Tim Kelleher', 'Silvie Laguna', 'Natasha Bea...","['s', 'philosophy', 'skilled', 'kidnapping', '..."
1,"['rescue', 'future', 'spacecraft', 'race again...","['Adventure', 'Drama', 'Science Fiction']","['Jeff Hephner', 'William Devane', 'Elyes Gabe...","['thoughtful', 'use', 'scientist', 'quantum me..."
2,"['joker', 'sadism', 'chaos', 'secret identity'...","['Drama', 'Action', 'Crime', 'Thriller']","['Tommy Lister Jr.', 'Edison Chen', 'Beatrice ...","['reign', 'harvey', 'proves', 'partnership', '..."
3,"['future', 'society', 'culture clash', 'space ...","['Action', 'Adventure', 'Fantasy', 'Science Fi...","['Carvon Futrell', 'Joel David Moore', 'Jon Cu...","['space war', 'civilization', 'society', 'mari..."
4,"['new york city', 'superhero', 'shield', 'base...","['Science Fiction', 'Action', 'Adventure']","['Haneyuri', 'Nako Mizusawa', 'Marin', 'Rikako...","['s', 'world', 'begins', 'e', 'global safety',..."


In [13]:
array_df.iloc[0].progress_apply(type)

  0%|          | 0/4 [00:00<?, ?it/s]

keywords                 <class 'str'>
genres_list              <class 'str'>
Cast_list                <class 'str'>
all_combined_keywords    <class 'str'>
Name: 0, dtype: object

The `array` are in `str` format. So we will need to change them back.

In [14]:
array_df = array_df.applymap(ast.literal_eval)
array_df.head()

,keywords,genres_list,Cast_list,all_combined_keywords
0,"[rescue, mission, dream, airplane, paris, fran...","[Action, Science Fiction, Adventure]","[Tim Kelleher, Silvie Laguna, Natasha Beaumont...","[s, philosophy, skilled, kidnapping, manipulat..."
1,"[rescue, future, spacecraft, race against time...","[Adventure, Drama, Science Fiction]","[Jeff Hephner, William Devane, Elyes Gabel, To...","[thoughtful, use, scientist, quantum mechanics..."
2,"[joker, sadism, chaos, secret identity, crime ...","[Drama, Action, Crime, Thriller]","[Tommy Lister Jr., Edison Chen, Beatrice Rosen...","[reign, harvey, proves, partnership, batman, n..."
3,"[future, society, culture clash, space travel,...","[Action, Adventure, Fantasy, Science Fiction]","[Carvon Futrell, Joel David Moore, Jon Curry, ...","[space war, civilization, society, marine, mis..."
4,"[new york city, superhero, shield, based on co...","[Science Fiction, Action, Adventure]","[Haneyuri, Nako Mizusawa, Marin, Rikako Sakata...","[s, world, begins, e, global safety, recruitme..."


In [15]:
for attribute in tqdm(list(array_df.keys())):
    df[attribute] = array_df[attribute]

print(array_df.iloc[0].apply(type))
df[['keywords', 'genres_list', 'Cast_list', 'all_combined_keywords']].head()

  0%|          | 0/4 [00:00<?, ?it/s]

keywords                 <class 'list'>
genres_list              <class 'list'>
Cast_list                <class 'list'>
all_combined_keywords    <class 'list'>
Name: 0, dtype: object


,keywords,genres_list,Cast_list,all_combined_keywords
0,"[rescue, mission, dream, airplane, paris, fran...","[Action, Science Fiction, Adventure]","[Tim Kelleher, Silvie Laguna, Natasha Beaumont...","[s, philosophy, skilled, kidnapping, manipulat..."
1,"[rescue, future, spacecraft, race against time...","[Adventure, Drama, Science Fiction]","[Jeff Hephner, William Devane, Elyes Gabel, To...","[thoughtful, use, scientist, quantum mechanics..."
2,"[joker, sadism, chaos, secret identity, crime ...","[Drama, Action, Crime, Thriller]","[Tommy Lister Jr., Edison Chen, Beatrice Rosen...","[reign, harvey, proves, partnership, batman, n..."
3,"[future, society, culture clash, space travel,...","[Action, Adventure, Fantasy, Science Fiction]","[Carvon Futrell, Joel David Moore, Jon Curry, ...","[space war, civilization, society, marine, mis..."
4,"[new york city, superhero, shield, based on co...","[Science Fiction, Action, Adventure]","[Haneyuri, Nako Mizusawa, Marin, Rikako Sakata...","[s, world, begins, e, global safety, recruitme..."


### Data Cleaning

In [16]:
df['status'].value_counts()

status
Released           1050747
In Production         7984
Post Production       6903
Planned               6029
Rumored                301
Canceled               291
Name: count, dtype: int64

We only care about movies than have been released and generated revenue to measure success with.

In [17]:
df = df[df['status'] == 'Released']
df.shape

(1050747, 42)

In [18]:
df.isnull().sum()

id                               0
title                            0
vote_average                     0
vote_count                       0
status                           0
release_date                135381
revenue                          0
runtime                          0
adult                            0
backdrop_path               761063
budget                           0
homepage                    937402
imdb_id                     469736
original_language                0
original_title                   0
overview                    197271
popularity                       0
poster_path                 305519
tagline                     901100
production_companies        566460
production_countries        448415
spoken_languages            434121
keywords                         0
release_year                135381
Director                         0
AverageRating              1039145
Poster_Link                1048033
Certificate                1021365
IMDB_Rating         

IMDB Dataset Attribute Analysis

Potentially Unnecessary Attributes

1. AverageRating (1,060,504 nulls)
- **Explanation:** This attribute is almost entirely null. It's redundant with `vote_average` and `IMDB_Rating`, which have no null values.

2. Poster_Link (1,069,485 nulls)
- **Explanation:** Nearly all entries are null. The `poster_path` attribute likely serves a similar purpose with fewer null values.

3. Certificate (1,042,420 nulls)
- **Explanation:** Mostly null. If certification information is crucial, it might be better to source it from a more complete dataset.

4. IMDB_Rating (1,041,943 nulls)
- **Explanation:** Largely null and likely redundant with `vote_average`.

5. Meta_score (1,069,924 nulls)
- **Explanation:** Almost entirely null. If Metacritic scores are desired, a more complete source should be used.

6. Music_Composer (916,779 nulls)
- **Explanation:** Largely null. This information might be included in the `Producers` or `Cast_list` attributes.

7. Director_of_Photography (735,402 nulls)
- **Explanation:** Mostly null. This level of detail might not be necessary for general analysis.

8. homepage (956,736 nulls)
- **Explanation:** Mostly null and not crucial for most analyses.

9. tagline (919,472 nulls)
- **Explanation:** Largely null and often not essential for analysis. 

8. Producers (646,947 nulls)
- **Explanation:** Mostly null.

9. Writer (425,675 nulls)
- **Explanation:** While it has fewer nulls than some other attributes, it's still mostly empty and might be redundant with information in Cast_list or overview.

<br>

Attributes to Consider Keeping Despite High Null Counts

1. production_countries (456,376 nulls)
- While it has many nulls, this information could be valuable for regional analysis.

2. spoken_languages (440,257 nulls)
- This could be important for language-based analysis, despite the high null count.

3. imdb_id (482,891 nulls)
- This could be useful for linking to external IMDB data, even though it has many nulls.

4. backdrop_path (779,429 nulls)
- While mostly null, it might be useful for visual representations in some applications.

5. release_date and release_year (both 150,556 nulls)
- These are important for temporal analysis, and the null count is relatively low compared to the dataset size.

6. Star1, Star2, Star3, Star4 (all have 1,041,942 or more nulls)
- **Explanation:** These are mostly null and redundant with the `Cast_list` attribute, which has no null values. But are important in determining the potential popularity of a movie based on its star-studded cast.

In [19]:
# Assuming your DataFrame is named 'imdb_df'

unnecessary_attributes = [
    'AverageRating',
    'Poster_Link',
    'Certificate',
    'IMDB_Rating',
    'Meta_score',
    'Music_Composer',
    'Director_of_Photography',
    'homepage',
    'tagline',
    'Producers',
    'Writer'
]

# Drop the unnecessary columns
df2 = df.drop(columns=unnecessary_attributes)
df2.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_companies', 'production_countries',
       'spoken_languages', 'keywords', 'release_year', 'Director', 'Star1',
       'Star2', 'Star3', 'Star4', 'genres_list', 'Cast_list',
       'overview_sentiment', 'all_combined_keywords'],
      dtype='object')

In [20]:
df2.isnull().sum()

id                             0
title                          0
vote_average                   0
vote_count                     0
status                         0
release_date              135381
revenue                        0
runtime                        0
adult                          0
backdrop_path             761063
budget                         0
imdb_id                   469736
original_language              0
original_title                 0
overview                  197271
popularity                     0
poster_path               305519
production_companies      566460
production_countries      448415
spoken_languages          434121
keywords                       0
release_year              135381
Director                       0
Star1                    1020895
Star2                    1048033
Star3                    1048033
Star4                    1048033
genres_list                    0
Cast_list                      0
overview_sentiment             0
all_combin

In [21]:
df[['release_date', 'release_year']].head()

,release_date,release_year
0,2010-07-15,2010.0
1,2014-11-05,2014.0
2,2008-07-16,2008.0
3,2009-12-15,2009.0
4,2012-04-25,2012.0


In [22]:
df[['release_date', 'release_year']].tail()

,release_date,release_year
1072250,2017-07-13,2017.0
1072251,2019-06-12,2019.0
1072252,2020-06-15,2020.0
1072253,1964-02-02,1964.0
1072254,1921-11-27,1921.0


In [23]:
df[['release_date', 'release_year']].isnull().sum()

release_date    135381
release_year    135381
dtype: int64

In [24]:
df[(df['release_date'].isnull()) & (df['release_year'].isnull())].isnull().sum()

id                              0
title                           0
vote_average                    0
vote_count                      0
status                          0
release_date               135381
revenue                         0
runtime                         0
adult                           0
backdrop_path              127668
budget                          0
homepage                   122583
imdb_id                    100430
original_language               0
original_title                  0
overview                    44401
popularity                      0
poster_path                 90915
tagline                    119193
production_companies       125831
production_countries       123146
spoken_languages           112100
keywords                        0
release_year               135381
Director                        0
AverageRating              134490
Poster_Link                135164
Certificate                132927
IMDB_Rating                132873
Meta_score    

The time data of this dataset is important to us, and thus we will not be able to use those rows without a date.

In [25]:
df2.dropna(subset=['release_date'], inplace=True)
df2.dropna(subset=['release_year'], inplace=True)
df2.shape

(915366, 31)

In [26]:
df2[['Star1', 'Star2', 'Star3', 'Star4']].head()

,Star1,Star2,Star3,Star4
0,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe
1,Matthew McConaughey,Anne Hathaway,Jessica Chastain,Mackenzie Foy
2,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine
3,Sam Worthington,Zoe Saldana,Sigourney Weaver,Michelle Rodriguez
4,Robert Downey Jr.,Chris Evans,Scarlett Johansson,Jeremy Renner


In [27]:
df2[['Star1', 'Star2', 'Star3', 'Star4']].tail()

,Star1,Star2,Star3,Star4
1072250,NaN,NaN,NaN,NaN
1072251,NaN,NaN,NaN,NaN
1072252,NaN,NaN,NaN,NaN
1072253,NaN,NaN,NaN,NaN
1072254,NaN,NaN,NaN,NaN


This identifies if there are any popular/famous movie starts starring in the movie. So it is ok to have `null` values.

In [28]:
df2['Star1'] = df2['Star1'].fillna('Unknown')
df2['Star2'] = df2['Star2'].fillna('Unknown')
df2['Star3'] = df2['Star3'].fillna('Unknown')
df2['Star4'] = df2['Star4'].fillna('Unknown')
df2[['Star1', 'Star2', 'Star3', 'Star4']].tail()

,Star1,Star2,Star3,Star4
1072250,Unknown,Unknown,Unknown,Unknown
1072251,Unknown,Unknown,Unknown,Unknown
1072252,Unknown,Unknown,Unknown,Unknown
1072253,Unknown,Unknown,Unknown,Unknown
1072254,Unknown,Unknown,Unknown,Unknown


In [29]:
df2[['production_countries', 'production_companies']].head()

,production_countries,production_companies
0,"United Kingdom, United States of America","Legendary Pictures, Syncopy, Warner Bros. Pict..."
1,"United Kingdom, United States of America","Legendary Pictures, Syncopy, Lynda Obst Produc..."
2,"United Kingdom, United States of America","DC Comics, Legendary Pictures, Syncopy, Isobel..."
3,"United States of America, United Kingdom","Dune Entertainment, Lightstorm Entertainment, ..."
4,United States of America,Marvel Studios


In [30]:
df2[['production_countries', 'production_companies']].tail()

,production_countries,production_companies
1072250,NaN,NaN
1072251,Poland,PWSFTviT
1072252,South Korea,Korea National University of Arts
1072253,Greece,Spentzos Films
1072254,NaN,NaN


In [31]:
df2[(df2['production_companies'].isnull()) & (df2['production_countries'].isnull())].isnull().sum()

id                            0
title                         0
vote_average                  0
vote_count                    0
status                        0
release_date                  0
revenue                       0
runtime                       0
adult                         0
backdrop_path            227170
budget                        0
imdb_id                  145139
original_language             0
original_title                0
overview                  50486
popularity                    0
poster_path               91777
production_companies     267197
production_countries     267197
spoken_languages         179303
keywords                      0
release_year                  0
Director                      0
Star1                         0
Star2                         0
Star3                         0
Star4                         0
genres_list                   0
Cast_list                     0
overview_sentiment            0
all_combined_keywords         0
dtype: i

In [32]:
df2['production_companies'] = df2['production_companies'].fillna('Unknown')
df2['production_countries'] = df2['production_countries'].fillna('Unknown')
df2[['production_companies', 'production_countries']].tail()

,production_companies,production_countries
1072250,Unknown,Unknown
1072251,PWSFTviT,Poland
1072252,Korea National University of Arts,South Korea
1072253,Spentzos Films,Greece
1072254,Unknown,Unknown


In [33]:
df2.isnull().sum()

id                            0
title                         0
vote_average                  0
vote_count                    0
status                        0
release_date                  0
revenue                       0
runtime                       0
adult                         0
backdrop_path            633395
budget                        0
imdb_id                  369306
original_language             0
original_title                0
overview                 152870
popularity                    0
poster_path              214604
production_companies          0
production_countries          0
spoken_languages         322021
keywords                      0
release_year                  0
Director                      0
Star1                         0
Star2                         0
Star3                         0
Star4                         0
genres_list                   0
Cast_list                     0
overview_sentiment            0
all_combined_keywords         0
dtype: i

In [34]:
df2[['backdrop_path', 'poster_path']].head()

,backdrop_path,poster_path
0,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg
1,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg
2,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,/qJ2tW6WMUDux911r6m7haRef0WH.jpg
3,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg
4,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg


In [35]:
df2[['backdrop_path', 'poster_path']].tail()

,backdrop_path,poster_path
1072250,NaN,/tKnwO0Tc7p8gq1YffNwUPuXbl6f.jpg
1072251,/xfSlHmpbqOAHaWMVzyN3XXF4PyE.jpg,/uVRCFcgvyiPUqAHO1xYoeUqBss9.jpg
1072252,NaN,/9y6hHj6gTpGIilOgN2LhNh8XHIQ.jpg
1072253,/cIKFTIJQqhb9EX1YIgPGlm2voyK.jpg,/ekLcnTMKm3rwwCfzApullRbTT4R.jpg
1072254,NaN,NaN


On more intricate analysis, these attributes do not contribute to the project.

In [36]:
df2 = df2.drop(columns=['backdrop_path', 'poster_path'])
df2.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'budget', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'spoken_languages', 'keywords', 'release_year',
       'Director', 'Star1', 'Star2', 'Star3', 'Star4', 'genres_list',
       'Cast_list', 'overview_sentiment', 'all_combined_keywords'],
      dtype='object')

In [37]:
df2[['imdb_id', 'overview', 'spoken_languages']].head()

,imdb_id,overview,spoken_languages
0,tt1375666,"Cobb, a skilled thief who commits corporate es...","English, French, Japanese, Swahili"
1,tt0816692,The adventures of a group of explorers who mak...,English
2,tt0468569,Batman raises the stakes in his war on crime. ...,"English, Mandarin"
3,tt0499549,"In the 22nd century, a paraplegic Marine is di...","English, Spanish"
4,tt0848228,When an unexpected enemy emerges and threatens...,"English, Hindi, Russian"


In [38]:
df2[['imdb_id', 'overview', 'spoken_languages']].tail()

,imdb_id,overview,spoken_languages
1072250,NaN,Mahmoud and Nabila are married couple in their...,Arabic
1072251,tt10784416,"In order to confront her unfaithful partner, a...",Polish
1072252,NaN,A high school student. My best friend is pregn...,NaN
1072253,tt0231143,NaN,Greek
1072254,tt0403050,An Aesop’s Film Fables cartoon.,NaN


`imdb_id` is not useful for further research, while `overview` and `spoken_languages` are still useful.

In [39]:
df2 = df2.drop(columns=['imdb_id'])
df2['overview'] = df2['overview'].fillna('Unknown')
df2['spoken_languages'] = df2['spoken_languages'].fillna('Unknown')
df2.isnull().sum()

id                       0
title                    0
vote_average             0
vote_count               0
status                   0
release_date             0
revenue                  0
runtime                  0
adult                    0
budget                   0
original_language        0
original_title           0
overview                 0
popularity               0
production_companies     0
production_countries     0
spoken_languages         0
keywords                 0
release_year             0
Director                 0
Star1                    0
Star2                    0
Star3                    0
Star4                    0
genres_list              0
Cast_list                0
overview_sentiment       0
all_combined_keywords    0
dtype: int64

In [40]:
df2 = df2.drop_duplicates(subset='title')
df2.shape

(792403, 28)

In [41]:
df2['actor_list'] = df2.progress_apply(lambda x: [x['Star1'], x['Star2'], x['Star3'], x['Star4']] + x['Cast_list'], axis=1)
print(df2['actor_list'][0])

  0%|          | 0/792403 [00:00<?, ?it/s]

['Leonardo DiCaprio', 'Joseph Gordon-Levitt', 'Elliot Page', 'Ken Watanabe', 'Tim Kelleher', 'Silvie Laguna', 'Natasha Beaumont', 'Kraig Thornber', 'Jack Murray', 'Adam Cole', 'Claire Geare', 'Marion Cotillard', 'Magnus Nolan', 'Tai-Li Lee', 'Shannon Welles', 'Taylor Geare', 'Tom Berenger', 'Coralie Dedykere', 'Carl Gilliard', 'Miranda Nolan', 'Earl Cameron', 'Yuji Okumoto', 'Helena Cullinan', 'Nicolas Clerc', 'Andrew Pleavin', 'Alex Lombard', 'Mark Fleischmann', 'Michael Gaston', 'Marc Raducci', 'Jack Gilroy', 'Nicole Pulliam', 'Shelley Lang', 'Lukas Haas', 'Russ Fega', 'Felix Scott', 'Ryan Hayward', 'Cillian Murphy', 'Dileep Rao', 'Jill Maddrell', 'Jean-Michel Dagory', 'Jason Tendell', 'Virgile Bramly', 'Tom Hardy', 'Tohoru Masamune', 'Michael Caine', 'Talulah Riley', 'Angela Nathenson', 'Lisa Reynolds', 'Peter Basham', 'Daniel Girondeaud', 'Johnathan Geare', 'Pete Postlethwaite']


In [42]:
df2 = df2[df2['adult'] == False]
df2 = df2[df2['status'] == 'Released']
df2.shape

(709920, 29)

In [43]:
df2['director'] = df2['Director'].progress_apply(lambda x: x.split(', '))
df2['genre'] = df2['genres_list']

  0%|          | 0/709920 [00:00<?, ?it/s]

In [44]:
df2.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'budget', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'spoken_languages', 'keywords', 'release_year',
       'Director', 'Star1', 'Star2', 'Star3', 'Star4', 'genres_list',
       'Cast_list', 'overview_sentiment', 'all_combined_keywords',
       'actor_list', 'director', 'genre'],
      dtype='object')

In [45]:
df2['production_companies'] = df2['production_companies'].progress_apply(lambda x: x.split(', '))
df2['production_countries'] = df2['production_countries'].progress_apply(lambda x: x.split(', '))

  0%|          | 0/709920 [00:00<?, ?it/s]

  0%|          | 0/709920 [00:00<?, ?it/s]

In [46]:
df2.drop(columns=['overview', 'original_title', 'all_combined_keywords', 'adult', 'status', 'release_year', 'spoken_languages', 'Star1', 'Star2', 'Star3', 'Star4', 'Director', 'Cast_list', 'genres_list'], inplace=True)
df2.head()

,id,title,vote_average,vote_count,release_date,revenue,runtime,budget,original_language,popularity,production_companies,production_countries,keywords,overview_sentiment,actor_list,director,genre
0,27205,Inception,8.364,34495,2010-07-15,825532764,148,160000000,en,83.952,"[Legendary Pictures, Syncopy, Warner Bros. Pic...","[United Kingdom, United States of America]","[rescue, mission, dream, airplane, paris, fran...",-0.011111,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elli...",[Christopher Nolan],"[Action, Science Fiction, Adventure]"
1,157336,Interstellar,8.417,32571,2014-11-05,701729206,169,165000000,en,140.241,"[Legendary Pictures, Syncopy, Lynda Obst Produ...","[United Kingdom, United States of America]","[rescue, future, spacecraft, race against time...",0.045455,"[Matthew McConaughey, Anne Hathaway, Jessica C...",[Christopher Nolan],"[Adventure, Drama, Science Fiction]"
2,155,The Dark Knight,8.512,30619,2008-07-16,1004558444,152,185000000,en,130.643,"[DC Comics, Legendary Pictures, Syncopy, Isobe...","[United Kingdom, United States of America]","[joker, sadism, chaos, secret identity, crime ...",0.025000,"[Christian Bale, Heath Ledger, Aaron Eckhart, ...",[Christopher Nolan],"[Drama, Action, Crime, Thriller]"
3,19995,Avatar,7.573,29815,2009-12-15,2923706026,162,237000000,en,79.932,"[Dune Entertainment, Lightstorm Entertainment,...","[United States of America, United Kingdom]","[future, society, culture clash, space travel,...",0.041667,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]"
4,24428,The Avengers,7.710,29166,2012-04-25,1518815515,143,220000000,en,98.082,[Marvel Studios],[United States of America],"[new york city, superhero, shield, based on co...",0.025000,"[Robert Downey Jr., Chris Evans, Scarlett Joha...",[Joss Whedon],"[Science Fiction, Action, Adventure]"


In [47]:
df2_1 = df2.copy()

We will also further limit our dataset to contain the movies that have been voted for more than 100 times, just so that we know that they are relevant.

In [48]:
print(df2.shape)
df2 = df2[df2['vote_count'] >= 100]
print(df2.shape)

(709920, 17)
(17328, 17)


We also want the movies to be relevant to this day and age. Thus, we will keep the movies from 2000 onwards.

In [49]:
print(df2.shape)
df2 = df2[df2['release_date'] >= '2000-01-01']
print(df2.shape)

(17328, 17)
(12173, 17)


In [57]:
print(df2_1.shape)
df2_1 = df2_1[df2_1['release_date'] >= '2000-01-01']
print(df2_1.shape)

(709920, 17)
(454347, 17)


In [50]:
df3 = df2
df3.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'release_date', 'revenue',
       'runtime', 'budget', 'original_language', 'popularity',
       'production_companies', 'production_countries', 'keywords',
       'overview_sentiment', 'actor_list', 'director', 'genre'],
      dtype='object')

### Row Expansion

In [51]:
df3.shape

(12173, 17)

In [52]:
for i in tqdm(df3.columns):
    print(f'{i}: {type(df3[i][0])}, {df3[i][0]}')

  0%|          | 0/17 [00:00<?, ?it/s]

id: <class 'numpy.int64'>, 27205
title: <class 'str'>, Inception
vote_average: <class 'numpy.float64'>, 8.364
vote_count: <class 'numpy.int64'>, 34495
release_date: <class 'pandas._libs.tslibs.timestamps.Timestamp'>, 2010-07-15 00:00:00
revenue: <class 'numpy.int64'>, 825532764
runtime: <class 'numpy.int64'>, 148
budget: <class 'numpy.int64'>, 160000000
original_language: <class 'str'>, en
popularity: <class 'numpy.float64'>, 83.952
production_companies: <class 'list'>, ['Legendary Pictures', 'Syncopy', 'Warner Bros. Pictures']
production_countries: <class 'list'>, ['United Kingdom', 'United States of America']
keywords: <class 'list'>, ['rescue', 'mission', 'dream', 'airplane', 'paris', 'france', 'virtual reality', 'kidnapping', 'philosophy', 'spy', 'allegory', 'manipulation', 'car crash', 'heist', 'memory', 'architecture', 'los angeles', 'california', 'dream world', 'subconscious']
overview_sentiment: <class 'numpy.float64'>, -0.0111111111111111
actor_list: <class 'list'>, ['Leonardo

In [53]:
df3 = df3.explode('production_companies').explode('production_countries').explode('keywords').explode('actor_list').explode('director').explode('genre')
df3.shape

(43174834, 17)

In [54]:
df3.head()

,id,title,vote_average,vote_count,release_date,revenue,runtime,budget,original_language,popularity,production_companies,production_countries,keywords,overview_sentiment,actor_list,director,genre
0,27205,Inception,8.364,34495,2010-07-15,825532764,148,160000000,en,83.952,Legendary Pictures,United Kingdom,rescue,-0.011111,Leonardo DiCaprio,Christopher Nolan,Action
0,27205,Inception,8.364,34495,2010-07-15,825532764,148,160000000,en,83.952,Legendary Pictures,United Kingdom,rescue,-0.011111,Leonardo DiCaprio,Christopher Nolan,Science Fiction
0,27205,Inception,8.364,34495,2010-07-15,825532764,148,160000000,en,83.952,Legendary Pictures,United Kingdom,rescue,-0.011111,Leonardo DiCaprio,Christopher Nolan,Adventure
0,27205,Inception,8.364,34495,2010-07-15,825532764,148,160000000,en,83.952,Legendary Pictures,United Kingdom,rescue,-0.011111,Joseph Gordon-Levitt,Christopher Nolan,Action
0,27205,Inception,8.364,34495,2010-07-15,825532764,148,160000000,en,83.952,Legendary Pictures,United Kingdom,rescue,-0.011111,Joseph Gordon-Levitt,Christopher Nolan,Science Fiction


### Export Data

In [55]:
df2.to_csv('../data/imdb_tmdb_cleaned.csv', index=False)

In [56]:
df3.to_csv('../data/imdb_imdb_tableau.csv', index=False)

In [59]:
df2_1.to_csv('../data/imdb_tmdb_cleaned_w|o_filtering.csv', index=False)

In [104]:
# # Open the file in write mode
# with open('../data/imdb_tmdb_cleaned.csv', 'w', newline='', encoding='utf-8') as f:
#     # Write the header first
#     df3.iloc[:0].to_csv(f, index=False)
    
#     # Use tqdm to track the progress of writing rows to CSV
#     for i in tqdm(range(0, len(df3))):
#         df3.iloc[[i]].to_csv(f, header=False, index=False)